In [ ]:
import os
import json
import numpy as np
from tqdm import tqdm
import pandas as pd

In [ ]:
task_root = 'PATH_TO_DATASET_ROOT'
in_hand_cam_id = 'cam_043322070878'
fixed_cam_ids = ['cam_750612070851']
# user_id and task_id can be used to filter the dataset
user_id = None
task_id = None

In [ ]:
def validate_camera(task_path, cam_id, metadata):
    if 'bad_calib_view' in metadata and cam_id[4:] in metadata['bad_calib_view']:
        return False
    color_dir = os.path.join(task_path, cam_id, 'color')
    tcp_dir = os.path.join(task_path, cam_id, 'tcp')
    gripper_info_dir = os.path.join(task_path, cam_id, 'gripper_info')
    if not os.path.exists(color_dir):
        return False
    if not os.path.exists(tcp_dir):
        return False
    if not os.path.exists(gripper_info_dir):
        return False
    frame_ids = os.listdir(color_dir)
    if len(frame_ids) <= 10:
        return False
    if not os.path.exists(os.path.join(task_path, cam_id, 'timestamps.npy')):
        frame_ids = [int(f[:-4]) for f in frame_ids]
        frame_ids = sorted(frame_ids)
        np.save(os.path.join(task_path, cam_id, 'timestamps.npy'), {'color': frame_ids})

    return True

In [ ]:
arr_all = []
task_ids = os.listdir(task_root)
task_ids = [x for x in task_ids if x[:4] == 'task']
for task in tqdm(task_ids, desc=f'Loading...'):
    task_path = os.path.join(task_root, task)

    parts = task.split('_')
    if user_id is not None and int(parts[3]) != user_id:
        continue
    if task_id is not None and int(parts[1]) != task_id:
        continue
    
    files = os.listdir(task_path)
    fixed_cams = [f for f in files if f in fixed_cam_ids]

    with open(os.path.join(task_path, 'metadata.json')) as f:
        metadata = json.load(f)
    if 'finish_time' not in metadata or metadata['finish_time'] == -1:
        continue
    if 'rating' not in metadata or metadata['rating'] <= 1:
        continue
    
    cams = []
    cam_types = []
    if validate_camera(task_path, in_hand_cam_id, metadata):
        cams.append(in_hand_cam_id)
        cam_types.append('in_hand')
    for cam_id in fixed_cams:
        if validate_camera(task_path, cam_id, metadata):
            cams.append(cam_id)
            cam_types.append('fixed')
    if len(cams) == 0:
        continue

    start_timestamps = []
    end_timestamps = []
    cam_timestamps = {}
    for cam_id in cams:
        timestamp_path = os.path.join(task_path, cam_id, 'timestamps.npy')
        timestamps = np.array(np.load(timestamp_path, allow_pickle=True).item()['color'])
        cam_timestamps[cam_id] = timestamps
        start_timestamps.append(timestamps[1])
        end_timestamps.append(timestamps[-2])
    max_start_timestamp = max(start_timestamps) - 50
    min_end_timestamp = min(end_timestamps) + 50
    min_end_timestamp = min(min_end_timestamp, metadata['finish_time'] + 50)

    demo_len = min_end_timestamp - max_start_timestamp
    if demo_len < 1 * 1000 or demo_len > 120 * 1000: # only keep video with reasonable length
        continue
    
    for cam_id, cam_type in zip(cams, cam_types):
        idx = np.flatnonzero((cam_timestamps[cam_id] >= max_start_timestamp) & (cam_timestamps[cam_id] <= min_end_timestamp))

        arr_all.append([
            task_path,
            int(parts[1]),  # task
            int(parts[3]),  # user
            int(parts[5]),  # scene
            int(parts[7]),  # config
            cam_id,
            cam_type,
            max_start_timestamp,
            min_end_timestamp,
            idx[0],
            idx[-1],
            cam_timestamps[cam_id][idx[0]],
        ])

In [ ]:
df = pd.DataFrame(columns=['path', 'task', 'user', 'scene', 'config', 'cam_id', 'cam_type', 'start_timestamp', 'end_timestamp', 'start_frame_idx', 'end_frame_idx', 'start_frame_timestamp'], data=arr_all)
df

In [ ]:
df.to_csv('dataset.csv', sep=' ', index=False)